# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'


# Then, this code will automatically setup your 
import os
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + ':' + 
                     str(port) + '/' + 
                     database)

In [2]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [3]:
url = 'https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326'

### BEGIN SOLUTION
df = pd.read_csv(url)
df.to_sql(MYTABLE, conn, if_exists='replace')
### END SOLUTION

In [5]:
dxyz.shape

(323, 22)

In [9]:
dxyz.columns

Index(['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'],
      dtype='object')

In [10]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

In [11]:
df.head()

,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,Business_ZIP_Code,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,95501,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,94545,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,94558,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,95642,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,94114,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019


## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [13]:
df = pd.read_csv('untitled.txt', '\t')

In [16]:
df.to_sql('population', conn)

In [32]:
maxpop = pd.read_sql_query("""
  SELECT m.*, p.*
  FROM {} m JOIN population p ON m.Business_ZIP_Code = p.Zip
  ORDER BY p.population DESC
  """.format(MYTABLE), conn)

maxzip = int(maxpop.loc[0,'Business_ZIP_Code'])
print(f'The zip with the most providers is {maxzip}')

maxnpis = list(maxpop[maxpop['Business_ZIP_Code'] == maxzip]['NPI'])
print(f'The NPIs in that zip are {maxnpis}')

The zip with the most providers is 90650
The NPIs in that zip are [1194016923]


In [33]:
maxnpis

[1194016923]

In [34]:
assert(type(answer) == list)

### BEGIN HIDDEN TESTS
assert(answer == ['1194016923'])
### END HIDDEN TESTS

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [45]:
### BEGIN SOLUTION
answer = pd.read_sql_query("""
  SELECT z.Medicaid_EP_Hospital_Type, AVG(z.Payments / p.population) as Avg_Pay_per_Capita
  FROM (
      SELECT m.Medicaid_EP_Hospital_Type, m.Business_ZIP_Code, sum(Total_payments) as Payments
      FROM {} m
      GROUP BY m.Medicaid_EP_Hospital_Type, m.Business_ZIP_Code
  ) z JOIN population p ON z.Business_ZIP_Code = p.Zip
  GROUP BY
    z.Medicaid_EP_Hospital_type
  """.format(MYTABLE), conn)
### END SOLUTIONS


In [46]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


In [47]:
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,329.550946


## Now using Pandas instead of SQL

In [48]:
### BEGIN SOLUTION
total_pay_per_zip = df.groupby(['Medicaid_EP_Hospital_Type','Zip_Code'])['Total_payments'].sum().reset_index()
total_pay_per_zip['Zip_Code'] = total_pay_per_zip['Zip_Code'].astype(str)

populations = pd.read_sql_query("SELECT * FROM population", conn)
populations['zipcode'] = populations['zipcode'].astype(str)

merged = total_pay_per_zip.merge(populations, how='left', left_on='Zip_Code', right_on='zipcode')

merged['Avg_Pay_per_Capita'] = merged['Total_payments'] / merged['population']

answer = merged.groupby('Medicaid_EP_Hospital_Type')['Avg_Pay_per_Capita'].mean().reset_index()

answer
### END SOLUTIONS


KeyError: 'Medicaid_EP_Hospital_Type'